In [9]:
# %matplotlib widget
# %load_ext autoreload
# %autoreload 2
import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
)
from amftrack.pipeline.launching.run_super import (
    run_parallel_transfer,
    find_state,
    directory_scratch,
    directory_project,
    directory_archive,
)
import dropbox
from amftrack.util.dbx import upload, zip_file
from subprocess import call
from tqdm.autonotebook import tqdm
from decouple import Config, RepositoryEnv
from amftrack.util.dbx import (
    upload_folders,
    get_dropbox_folders,
    load_dbx,
    download,
    get_dropbox_folders_new,
)

DOTENV_FILE = os.path.dirname(os.path.dirname(os.path.abspath(""))) + "/local.env"
env_config = Config(RepositoryEnv(DOTENV_FILE))

In [ ]:
all_folders_drop_new = get_dropbox_folders_new("/DATA/PRINCE", False)

sizes:   0%|          | 0/16804 [00:00<?, ?it/s]

In [21]:
select_new = all_folders_drop_new.loc[all_folders_drop_new["size"] < 0.6]

In [22]:
all_folders_drop2 = get_dropbox_folders("/DATA/PRINCE_TEMP", True)

In [23]:
select_old = all_folders_drop2.loc[all_folders_drop2["size"] < 0.6]

In [24]:
select_new.loc[select_new["folder"].isin(select_old["folder"]) == False]

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
7522,20220415_1119_Plate05,0.593189,2022-05-20 18:32:12,data/prince/931_20220402/20220415_1119_plate05,931,20220402,931_20220402
7541,20220424_1925_Plate05,0.330101,2022-05-20 17:44:33,data/prince/931_20220402/20220424_1925_plate05,931,20220402,931_20220402
7566,20220424_1525_Plate05,0.350158,2022-05-20 16:42:41,data/prince/931_20220402/20220424_1525_plate05,931,20220402,931_20220402
7602,20220424_1125_Plate05,0.350141,2022-05-20 14:58:01,data/prince/931_20220402/20220424_1125_plate05,931,20220402,931_20220402
7641,20220425_0325_Plate05,0.360051,2022-05-20 13:18:46,data/prince/931_20220402/20220425_0325_plate05,931,20220402,931_20220402
7645,20220424_0325_Plate05,0.369395,2022-05-20 13:08:47,data/prince/931_20220402/20220424_0325_plate05,931,20220402,931_20220402
7666,20220425_0725_Plate05,0.47389,2022-05-20 12:12:58,data/prince/931_20220402/20220425_0725_plate05,931,20220402,931_20220402
7674,20220415_0919_Plate05,0.593038,2022-05-20 11:56:38,data/prince/931_20220402/20220415_0919_plate05,931,20220402,931_20220402
7680,20220424_0725_Plate05,0.350515,2022-05-20 11:42:47,data/prince/931_20220402/20220424_0725_plate05,931,20220402,931_20220402
7683,20220415_0719_Plate05,0.592977,2022-05-20 11:34:42,data/prince/931_20220402/20220415_0719_plate05,931,20220402,931_20220402


In [25]:
all_folders_drop_new.loc[
    all_folders_drop_new["folder"].isin(all_folders_drop2["folder"]) == False
]

,folder,size,change_date,tot_path_drop,Plate,CrossDate,unique_id
0,20220330_2140_Plate14,0.843886,2022-05-30 11:18:56,data/prince/889_20220325/20220330_2140_plate14,889,20220325,889_20220325
1,20220330_2128_Plate10,0.900234,2022-05-30 11:14:51,data/prince/983_20220321/20220330_2128_plate10,983,20220321,983_20220321
2,20220328_1842_Plate15,0.852564,2022-05-30 11:14:48,data/prince/938_20220325/20220328_1842_plate15,938,20220325,938_20220325
3,20220329_0009_Plate03,0.838905,2022-05-30 11:12:37,data/prince/838_20220325/20220329_0009_plate03,838,20220325,838_20220325
4,20220330_0649_Plate07,0.887398,2022-05-30 11:10:30,data/prince/865_20220307/20220330_0649_plate07,865,20220307,865_20220307
...,...,...,...,...,...,...,...
15251,20220317_0839_Plate14,0.883615,2022-05-18 21:30:40,data/prince/976_20220308/20220317_0839_plate14,976,20220308,976_20220308
15252,20220315_2208_Plate01,0.908231,2022-05-18 21:28:38,data/prince/866_20220315/20220315_2208_plate01,866,20220315,866_20220315
15253,20220315_1145_Plate01,0.902252,2022-05-18 21:24:35,data/prince/1007_20220301/20220315_1145_plate01,1007,20220301,1007_20220301
15254,20220316_0438_Plate14,0.870648,2022-05-18 21:24:32,data/prince/976_20220308/20220316_0438_plate14,976,20220308,976_20220308


In [25]:
dir_drop = "DATA/PRINCE"
all_folders_origin = get_current_folders(directory_origin, local=True)

folders = all_folders_origin.loc[all_folders_origin["Plate"] == "931"]

In [23]:
folders

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/skeleton_masked_compressed.mat,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder
43,/mnt/sun-temp/TEMP/PRINCE_syncing/20220416_033...,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,False,False,"16.04.2022, 03:36:",20220416_0336_Plate05
50,/mnt/sun-temp/TEMP/PRINCE_syncing/20220428_172...,931,22,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,False,False,"28.04.2022, 17:28:",20220428_1728_Plate22
223,/mnt/sun-temp/TEMP/PRINCE_syncing/20220412_091...,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,False,False,"12.04.2022, 09:10:",20220412_0910_Plate05
273,/mnt/sun-temp/TEMP/PRINCE_syncing/20220411_231...,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,False,False,"11.04.2022, 23:10:",20220411_2310_Plate05
293,/mnt/sun-temp/TEMP/PRINCE_syncing/20220418_133...,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,False,False,"18.04.2022, 13:39:",20220418_1339_Plate05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8716,/mnt/sun-temp/TEMP/PRINCE_syncing/20220426_011...,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,False,False,"26.04.2022, 01:10:",20220426_0110_Plate05
8725,/mnt/sun-temp/TEMP/PRINCE_syncing/20220417_213...,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,False,False,"17.04.2022, 21:39:",20220417_2139_Plate05
8742,/mnt/sun-temp/TEMP/PRINCE_syncing/20220419_114...,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,False,False,"19.04.2022, 11:43:",20220419_1143_Plate05
8786,/mnt/sun-temp/TEMP/PRINCE_syncing/20220416_113...,931,5,'Carrot','A5','001P100N100C','Y','-',20220402,25,...,False,False,False,False,False,False,False,False,"16.04.2022, 11:36:",20220416_1136_Plate05


In [8]:
upload_folders(folders, dir_drop=dir_drop, delete=False, catch_exception=False)

transferred:   0%|          | 0/542 [00:00<?, ?it/s]

ApiError('7bb8e7b256664f4c86fc0df70885f74a', UploadError('path', UploadWriteFailed(reason=WriteError('insufficient_space', None), upload_session_id='pid_upload_session:ABIE-SJWPbKWbGkI6nHGs6pP9T9YF64ayGX5Gxqhr9jL8Yu0')))
ApiError('b52811cdb226469ebb78d652bb9e2100', UploadError('path', UploadWriteFailed(reason=WriteError('insufficient_space', None), upload_session_id='pid_upload_session:ABIE-SVPoYGCLhtb6imxGhE2rPnWhWjkIGROst5Pa6D0NOTI')))
ApiError('379b72af4fac4a2b86055f01a2b9348a', UploadError('path', UploadWriteFailed(reason=WriteError('insufficient_space', None), upload_session_id='pid_upload_session:ABIE-a-GD5Cix8lBaEsPSTl1AjWkiHuqDlw1hdUkmIZi4YSk')))


zipping:   0%|          | 0/152 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
old_folders.loc[old_folders["Plate"] == "'832            %Plate number'"]

,total_path,Plate,PrincePos,root,strain,medium,split,Pbait,CrossDate,Temperature,...,/Analysis/skeleton_pruned_compressed.mat,/Analysis/transform.mat,/Analysis/transform_corrupt.mat,/Analysis/skeleton_realigned_compressed.mat,/Analysis/nx_graph_pruned.p,/Analysis/nx_graph_pruned_width.p,/Analysis/nx_graph_pruned_labeled.p,date,folder,date_datetime


In [6]:
set(list(old_folders["Plate"].str.replace("[^0-9]", "")))

/home/ipausers/bisot/temp/ipykernel_443098/2083350065.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  set(list(old_folders['Plate'].str.replace("[^0-9]", "")))


{'031',
 '1',
 '1015',
 '1019',
 '1023',
 '1031',
 '1032',
 '1040',
 '1045',
 '1048',
 '1053',
 '1070',
 '12',
 '13',
 '16',
 '18',
 '2',
 '20',
 '21',
 '24',
 '3',
 '32',
 '33',
 '4',
 '66',
 '68',
 '824',
 '832',
 '871',
 '877',
 '887',
 '931',
 '961'}

In [4]:
directory = r"/mnt/sun-temp/TEMP/PRINCE_analysis/"

update_plate_info(directory, local=True)
all_folders = get_current_folders(directory, local=True)

analysed:   0%|          | 0/543 [00:00<?, ?it/s]

In [6]:
folders = all_folders

In [5]:
dir_drop = "test"

***FIND TOKEN***

In [76]:
import dropbox
from dropbox import DropboxOAuth2FlowNoRedirect

"""
Populate your app key in order to run this locally
"""
APP_KEY = "gm3xirr1cgdjfhi"

auth_flow = DropboxOAuth2FlowNoRedirect(
    APP_KEY, use_pkce=True, token_access_type="offline"
)

authorize_url = auth_flow.start()
print("1. Go to: " + authorize_url)
print('2. Click "Allow" (you might have to log in first).')
print("3. Copy the authorization code.")
auth_code = input("Enter the authorization code here: ").strip()

try:
    oauth_result = auth_flow.finish(auth_code)
except Exception as e:
    print("Error: %s" % (e,))
    exit(1)

with dropbox.Dropbox(
    oauth2_refresh_token=oauth_result.refresh_token, app_key=APP_KEY
) as dbx:
    print(dbx)
    # # p = dropbox.common.PathRoot.namespace_id("<TEAM_FOLDER_ID>")
    # dbx = dbx.with_path_root(p)
    print(dbx, "Successfully set up client!")

1. Go to: https://www.dropbox.com/oauth2/authorize?response_type=code&client_id=gm3xirr1cgdjfhi&token_access_type=offline&code_challenge=1F8bJvz6vGuWhdKnTF4Jq4PsuJbQBJo0w9QdkBNDMDs&code_challenge_method=S256
2. Click "Allow" (you might have to log in first).
3. Copy the authorization code.


Enter the authorization code here:  x9vxVXoO1WcAAAAAAAAB-Wqfee0rzWhTmzEvIfh0ZTg


<dropbox.dropbox_client.Dropbox object at 0x7f1a470d1a00> Successfully set up client!


In [79]:
oauth_result.refresh_token

'JTGUKclu02UAAAAAAAAAATreksjJSYkv2dIaRosB73NCVqZMgRvnDAaBWQAjV5w-'